In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from transformers import MarianMTModel, MarianTokenizer
import numpy as np

# Task 1: Digit Recognition
class DigitRecognitionModel(nn.Module):
    def __init__(self):
        super(DigitRecognitionModel, self).__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(-1, 784)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Task 2: House Price Prediction
class HousePriceModel(nn.Module):
    def __init__(self):
        super(HousePriceModel, self).__init__()
        self.fc1 = nn.Linear(2, 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Task 3: Language Translation
class LanguageTranslationModel(nn.Module):
    def __init__(self):
        super(LanguageTranslationModel, self).__init__()
        self.model = MarianMTModel.from_pretrained('Helsinki-NLP/opus-mt-en-fr')
        self.tokenizer = MarianTokenizer.from_pretrained('Helsinki-NLP/opus-mt-en-fr')

    def forward(self, x):
        inputs = self.tokenizer(x, return_tensors='pt')
        outputs = self.model.generate(**inputs)
        translated_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        return translated_text

# Initialize the models
digit_model = DigitRecognitionModel()
house_price_model = HousePriceModel()
language_translation_model = LanguageTranslationModel()

# Train the digit recognition model
transform = transforms.Compose([transforms.ToTensor()])
train_dataset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(digit_model.parameters(), lr=0.001)
for epoch in range(10):
    for x, y in train_loader:
        optimizer.zero_grad()
        output = digit_model(x)
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

# Train the house price prediction model
data = torch.tensor([[1000, 2], [1200, 3], [1500, 4], [1800, 5], [2000, 6]], dtype=torch.float32)
labels = torch.tensor([200000, 250000, 300000, 350000, 400000], dtype=torch.float32).view(-1, 1)
criterion = nn.MSELoss()
optimizer = optim.Adam(house_price_model.parameters(), lr=0.001)
for epoch in range(100):
    optimizer.zero_grad()
    output = house_price_model(data)
    loss = criterion(output, labels)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

# Use the language translation model
sentence = 'Hello, how are you?'
translated_sentence = language_translation_model(sentence)
print(translated_sentence)
